In [1]:
import json

uniprot_file = "data\\UniProt.json"
protein_info_file = "data\\protein_information.json"
training_file = "data\\f_covae_training.json"

In [2]:
with open(protein_info_file, 'r') as file:
    proteins_raw = json.load(file)

proteins_prep = {}
for id in proteins_raw.keys():
    proteins_prep[id] = {}

In [3]:
def get_go_terms():
    global go_terms
    go_ids = []
    with open(uniprot_file, 'r') as file:
        data = json.load(file)
        results = data["results"]

    for result in results:
        if "uniProtKBCrossReferences" in result:
            references = result["uniProtKBCrossReferences"]
            for reference in references:
                if reference["database"] == "GO":
                    go_id = reference["id"]
                    if go_id in go_ids:
                        continue
                    go_ids.append(go_id)
                    go_term = reference["properties"][0]["value"]
                    match go_term[0]:
                        case "F": go_terms["mf"].append(go_id)
                        case "P": go_terms["bp"].append(go_id)
                        case "C": go_terms["cc"].append(go_id)
    
    #sorts go ids numerically within each aspect
    for aspect in go_terms.keys():
        go_terms[aspect].sort()

def check_go_terms():
    global go_terms
    for aspect, terms in go_terms.items():
        test_terms = []
        for term in terms:
            if term in test_terms: print("duplicate found"); continue
            test_terms.append(term)
        if len(test_terms) == len(go_terms[aspect]): print(f"all {aspect} terms accounted for. {len(test_terms)} {aspect} terms.")
        else: print(f"something went wrong: {aspect} terms counted = {len(test_terms)}, {aspect} terms expected = {len(go_terms[aspect])}")

def onehot_encode_go():
    global go_terms
    global proteins_raw
    global proteins_prep
    
    for aspect, terms in go_terms.items():
        for p_id, p_data in proteins_raw.items():
            p_encoded_go = [0] * len(go_terms[aspect])
            for p_go in p_data["go_terms"]:
                if p_go[0] in terms:
                    index = terms.index(p_go[0])
                    p_encoded_go[index] = 1
            proteins_prep[p_id]["go_terms"][aspect] = p_encoded_go

go_terms = {"mf": [], "cc": [], "bp": []}
for id in proteins_prep.keys():
    proteins_prep[id]["go_terms"] = {}

get_go_terms()
check_go_terms()
onehot_encode_go()

all mf terms accounted for. 697 mf terms.
all cc terms accounted for. 49 cc terms.
all bp terms accounted for. 392 bp terms.


In [ ]:
#test
test_protein_terms = proteins_raw["A0A0X8BZN2"]["go_terms"]
for term in test_protein_terms:
    if "F:" in term[1]: print(term[0]); print(go_terms["mf"].index(term[0]))
print()
c = -1
for num in proteins_prep["A0A0X8BZN2"]["go_terms"]["mf"]:
    c+=1
    if num == 1: print(c)